# Google Drive 마운트

`.pkl` 파일 백업을 위해 연결해둡니다.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 파라미터 설정

In [0]:
k=400 # size of codebook(kmeans)
imgsize = 256 #resize target
step_size = 8 #Step of SIFT
L = 2 #Level of SPM

# OpenCV SIFT 사용을 위한 재설치

- 최신 버전은 SIFT를 지원하지 않으므로 다운그레이드 합니다.

## Kmeans-GPU 설치

In [0]:
!pip uninstall -y opencv-python
!pip uninstall -y opencv-contrib-python

!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

!pip install kmc2

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 102kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 1.5MB/s 
     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.4MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252208 sha256=1220b38f791dcb5d08f672a9b9ad8c2f03279206e6332b1b80b7d80137658561
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


# 라이브러리 로드

In [0]:
import zipfile  # 데이터셋 압축 해제
import os       # 파일 경로
import cv2      # SIFT
import numpy as np  #이미지 행렬
import time     # 실행시간 측정
import pickle   # 결과물 저장
import math     # SPM에서 평균, 분산

from sklearn import preprocessing   #데이터셋 전처리
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split  # cross-validation
from sklearn.svm import SVC, LinearSVC  #SVM
from sklearn.model_selection import GridSearchCV    #파라미터 튜닝
from sklearn.cluster import KMeans, MiniBatchKMeans #Kmeans(소규모 메모리)
from sklearn.metrics import classification_report   #결과 측정
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  #LDA

import kmc2   #Kmeans-GPU
from pandas import read_csv   #데이터셋 라벨 로드
from google.colab import files  # 
from tqdm import tqdm_notebook as tqdm
from scipy.cluster.vq import vq

# Kaggle Token 로드

- github에 저장해둔 `kaggle.json` 파일을 다운로드합니다.

- kaggle API를 1.5.6 버전으로 재설치합니다.

In [0]:
!wget https://raw.githubusercontent.com/nhk9680/storage/master/asdf
!mv asdf kaggle.json

! pip uninstall -y kaggle
! pip install kaggle==1.5.6
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

--2019-12-14 02:02:32--  https://raw.githubusercontent.com/nhk9680/storage/master/asdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63 [text/plain]
Saving to: ‘asdf’

asdf                100%[===================>]      63  --.-KB/s    in 0s      

2019-12-14 02:02:33 (17.2 MB/s) - ‘asdf’ saved [63/63]

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=d6ba864339faac5a27e0ed4e789c6802875f7c45d0574764feec975e5b0ebf39
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


# 데이터셋 세팅

- kaggle에서 데이터셋 파일을 다운로드합니다.

In [0]:
! kaggle competitions download -c 2019-ml-finalproject

os.mkdir('../input')

zip_ref = zipfile.ZipFile("2019-ml-finalproject.zip", 'r')
zip_ref.extractall("../input")
zip_ref.close()

 63% 41.0M/65.0M [00:01<00:01, 21.0MB/s]
100% 65.0M/65.0M [00:01<00:00, 56.4MB/s]


## 데이터셋 전처리

- 이미지 파일을 동일한 크기의 흑백으로 가공합니다.

In [0]:
#load images
df_data = read_csv('/input/Label2Names.csv', header=None)

#train
train_dir = "/input/train"

train_data = []
train_label = []

i=1
for cls in os.listdir(train_dir):
  img_list = os.listdir(train_dir + "/" + cls)

  if cls == 'BACKGROUND_Google' :
    label = 102
    #continue
  else:
    label = (df_data.index[df_data[1]==cls]+1).tolist()[0]

  #category
  for img in img_list:

    image = cv2.imread(train_dir +'/'+cls+'/'+img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (imgsize, imgsize))

    train_data.append(gray)
    train_label.append(label)

#test
test_dir = "/input/testAll_v2"
test_data = []

test_filename = os.listdir(test_dir)  # kaggle 제출을 위해 저장

for img in test_filename:
  image = cv2.imread(test_dir+'/'+img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (imgsize, imgsize))
  test_data.append(gray)

# 특징점 추출

- SIFT 기법을 이용하여, 이미지에서 특징점을 추출합니다.

In [0]:
def extract_SIFT(img):
    disft_step_size = 2
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    _, descriptors = sift.compute(img, keypoints)
    
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

def computeSIFT(data):
    x = []
    for img in tqdm(data):
        des = extract_SIFT(img)
        x.append(des)
        
    return x

In [0]:
x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

# (-1, 128)
# same as np.reshape(x_train, (-1,128))
all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

### 세이브 파일 저장(옵션)

- 런타임 초기화 대비 앞서 진행했던 것들을 pickle로 묶어서 저장해둡니다.

  메모리 부족 등으로 터져나갔을때 유용합니다.

  데이터 용량이 크므로 직접 다운로드하지 말고, 구글 드라이브로 복사합니다.

  그러나 위 과정은 그렇게 오래 걸리지 않으므로 넘어가도 됩니다.

In [0]:
'''
preprocessed_data ={
    "train_data" : train_data,
    "train_label" : train_label,
    "test_filename" : test_filename,
    "test_index" : test_index,
    "x_train": x_train,
    "x_test" : x_test,
    "all_train_desc" : all_train_desc
}
with open("preprocessed_data.pkl", "wb") as f:
  pickle.dump(preprocessed_data, f)
'''

In [0]:
'''
for i in preprocessed_data:
  print(i, np.array(preprocessed_data[i]).shape)
'''

train_data (3060, 64, 64)
train_label (3060,)
test_data (1692, 64, 64)
test_index (1692,)
x_train (3060, 1024, 128)
x_test (1692, 1024, 128)
all_train_desc (3133440, 128)


In [0]:
#!cp preprocessed_data.pkl /content/drive/'My Drive'/preprocessed_data.pkl

In [0]:
'''
with open("preprocessed_data.pkl", "rb") as f:
  preprocessed_data = pickle.load(f)
  
train_data = preprocessed_data["train_data"]
train_label = preprocessed_data["train_label"]
test_data = preprocessed_data["test_data"]
x_train = preprocessed_data["x_train"]
x_test = preprocessed_data["x_test"]
all_train_desc = preprocessed_data["all_train_desc"]
'''

# Codebook 생성

- Kmeans 알고리즘을 이용하여 코드북을 생성합니다.
- Colab의 메모리 문제로 MiniBatchKMeans를 사용합니다.


In [0]:
#Kmeans-GPU

now = time.gmtime(time.time())
print(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec)
start=time.time()

seeding = kmc2.kmc2(all_train_desc, k)
#kmeans = KMeans(n_clusters = k, init=seeding)
kmeans = MiniBatchKMeans(n_clusters = k, init=seeding)

##########################################
kmeans.fit(all_train_desc)
##########################################

print(time.time()-start, "sec")

2019 12 14 11 3 37


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


425.86939907073975 sec


### 세이브 파일 저장(권장)

In [0]:
voc = kmeans.cluster_centers_

with open("voca_"+str(k)+".pkl", "wb") as f:
  pickle.dump(voc, f)

In [0]:
#files.download("./voca_"+str(k)+".pkl")
!cp voca_400.pkl /content/drive/'My Drive'/voca_400.pkl

In [0]:
with open("voca_"+str(k)+".pkl", "rb") as f:
  voc = pickle.load(f)
kmeans.cluster_centers_ = voc

# Spatial Pyramid Matching

- SPM을 이용하여 코드북에 대한 이미지의 히스토그램을 구합니다.

In [0]:
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])
                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    mean = np.mean(hist)
    hist = hist - mean
    hist /= dev
    return hist


# get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in tqdm(range(len(data))):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [0]:
train_histo = getHistogramSPM(2, train_data, kmeans, k)
test_histo = getHistogramSPM(2, test_data, kmeans, k)

### 세이브 파일 저장(옵션)

In [0]:
'''
histogram = {
    "train_histo" : train_histo,
    "test_histo" : test_histo
}
'''

In [0]:
'''
with open("histogram.pkl", "wb") as f:
  pickle.dump(histogram, f)
'''

In [0]:
'''
#!cp histogram_400_level3.pkl /content/drive/'My Drive'/histogram_400_level3.pkl
!cp /content/drive/'My Drive'/histogram_400_level3.pkl histogram_400_level3.pkl
'''

In [0]:
'''
with open("histogram_400_level3.pkl", "rb") as f:
  histogram = pickle.load(f)

train_histo = histogram["train_histo"]
test_histo = histogram["test_histo"]
'''

In [0]:
'''
lda = LinearDiscriminantAnalysis()
lda_train_histo = lda.fit_transform(train_histo, train_label)
'''

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Cross-validation

- 오버피팅을 방지하고 성능 체크를 하기 위해 데이터를 train/validation으로 나누어 사용합니다.

In [0]:
split_xtrain, split_xtest, split_ytrain, split_ytest = train_test_split(train_histo, train_label, test_size=0.25, random_state=42)

In [0]:
now = time.gmtime(time.time())
print(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec)
start=time.time()

svm = SVC(kernel='linear')
svm.fit(split_xtrain, split_ytrain)

print(time.time()-start, "sec")

2019 12 14 11 28 26
121.5873167514801 sec


In [0]:
print(classification_report(svm.predict(split_xtest), split_ytest))

              precision    recall  f1-score   support

           1       0.62      0.56      0.59         9
           2       1.00      0.86      0.92         7
           3       0.83      0.50      0.62        10
           4       0.86      1.00      0.92         6
           5       1.00      1.00      1.00        10
           6       1.00      0.70      0.82        10
           7       0.33      1.00      0.50         3
           8       0.12      0.06      0.08        17
           9       0.22      0.29      0.25         7
          10       0.00      0.00      0.00        16
          11       0.00      0.00      0.00         8
          12       0.75      0.25      0.38        12
          13       0.20      0.33      0.25         3
          14       0.83      0.38      0.53        13
          15       0.20      0.50      0.29         4
          16       0.60      0.50      0.55         6
          17       0.14      0.11      0.12         9
          18       0.57    

In [0]:
result = svm.predict(test_histo)

k=400 L=3 image_size=64 svm='linear' 35%

# GridSearch

- 파라미터 튜닝을 통해 성능 최적화를 진행합니다.

  단, 여기서 추천된 파라미터가 무조건 테스트 데이터셋에도  최고 성능으로 적용된다고 보장할 수는 없습니다.

In [0]:
param_grid = {
    'kernel' : ['linear', 'rbf'],
    'C': [1e-5,  1e-3,  1e-1, 0, 1e1, 1e3, 1e5],
    'gamma': [1e-5, 1e-3, 1e-1, 0, 1e+1, 1e+3, 1e+5], 
}

svc = SVC()
svm = GridSearchCV(svc, param_grid=param_grid)

now = time.gmtime(time.time())
print(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec)
start=time.time()

##########################################
svm.fit(split_xtrain, split_ytrain)
##########################################

print(time.time()-start, "sec")

In [0]:
print(classification_report(svm.predict(split_xtest), split_ytest))

In [0]:
result = svm.predict(test_histo)

# 결과 저장
- pandas를 이용하여 csv로 저장합니다.
- 가급적이면 파일명에 파라미터 정보를 기록합니다.

In [0]:
import pandas as pd

print(result.shape)
df = pd.DataFrame(result, columns=['Category'])
df.index = test_filename
df.index.name = 'Id'
df.to_csv('result_namhun_kim.csv',index=True, header=True)

(1692,)


# Kaggle에 제출

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f result_namhun_kim.csv -m "Namhun Kim"

100% 29.6k/29.6k [00:05<00:00, 5.09kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 

# ***SOTA 결과가 나왔다면 [파일] -> [버전 저장 및 고정] 또는 `Ctrl+M S`를 통해 세이브합니다.***